# Linear Regression


Content

- Linear Regression with One Feature

    - ...

- Linear Regression with Multiple Features

    - ...

ToDo:

- ...

Additional Material:

- ...

---
Linear Regression
- linear regression
- one variable
- residuals
- R2
- f-test, p-value

Feature Selection
- forwarnd/backward feature selection

Data Transormation
- outliers
- normality
- log transforamtion
- binning + one hot encoding

Missing Values
- binning imputation?
- missing indicator
---

---
## Central Concepts

All our inputs need to be **numeric** for linear regression.

**Linear Regression Assumptions**

   1. **Linearity**:\
   A linear correlation between the input and the target variable is assumed.\
   Otherwise, change the model.
   
   2. **Normality**:\
   Normal distributed input variables.\
   Otherwise, apply variable transformations.
   
   3. **No Multicorrealinity**\
   No correlation between input variables.
   Otherwise, the model can't identify the correct coeficients.

   4. **Homoscedasticity**:\
   Constant variance for the sample distribution.
   Otherwise, the model's robustness suffers.

**RMSE Metric**

$ rmse = \sqrt{\frac{1}{n}\sum (y_i - \hat{y}_i)^{2}}$

**R2 Metric**

$r^{2} = 1 - \frac{\sum (y_i - \hat{y}_i)^{2}}{\sum (y_i - \bar{y}_i)^{2}}$

- r2 < 0.3 --- none or very weak effect size
- 0.3 < r2 < 0.5 --- weak or low effect size
- 0.5 < r2 < 0.7 --- moderate effect size
- 0.7 < r2 < 1.0 --- strong effect size

**Z Score**

**Skew**

---

# Settings

In [ ]:
dark_plot_theme = True

if dark_plot_theme:
    plt.style.use('dark_background')


# pandas display settings

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import sys

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import d2_tweedie_score

#Scikit-learn is designed for prediction, while statsmodels is more suited for explanatory analysis.
import statsmodels.api as sm


In [ ]:
# reload ./utils.py

import importlib
import utils
importlib.reload(utils)
from utils import get_dichotomous

# Load Feature Data

assume: no nulls, no outliers (z>3)

In [ ]:
# prepare the directory and load the data

cwd = Path()

ipath = cwd / 'data'

ipath.mkdir(exist_ok=True)

ifile = ipath / 'features.csv'

data = pd.read_csv(ifile, index_col=['id'])

data.head(3)

In [ ]:
# TODO: check for null values

data.isnull().values.any() # REMOVE

---

# Linear Regression with One Feature

Let's try out a simple linear regression: predict the price from square meters.

In [ ]:
# define variable names

target = 'price'
features = ['square_meter']

variables = [target] + features

In [ ]:
# plot target distribution: price and log-price

figsize = (12, 5)
fig, axs = plt.subplots(1, 2, figsize=figsize)

# price distribution
d = data[target]
skew = d.skew()
title = f'skewness: {skew:0.2f}'
sns.histplot(d, bins=50, ax=axs[0]).set(title=title);

# log-price distribution
d = np.log(data[target]+1)
skew = d.skew()
title = f'skewness: {skew:0.2f}'
sns.histplot(d, bins=50, ax=axs[1]).set(title=title);

In [ ]:
# plot feature distribution: sqm and log-sqm

figsize = (12, 5)
fig, axs = plt.subplots(1, 3, figsize=figsize)

d = data[features[0]]
mask = data[f'imp_{features[0]}']+data[f'imp_z_{features[0]}'] < 1

# sqm distribution
skew = d.skew()
title = f'skewness: {skew:0.2f}'
sns.histplot(d, bins=50, ax=axs[0]).set(title=title);

# sqm distribution without imputation
d = d[mask]
skew = d.skew()
title = f'skewness: {skew:0.2f}'
sns.histplot(d, bins=50, ax=axs[1]).set(title=title);

# log sqm distribution
d = np.log(d+1)
skew = d.skew()
title = f'skewness: {skew:0.2f}'
sns.histplot(d, bins=50, ax=axs[2]).set(title=title);

In [ ]:
# define regression metrics

def rmse(*args, **kwargs):
    return mean_squared_error(*args, **kwargs, squared=False)

rmetrics = {}
rmetrics['r2_score'] = r2_score
rmetrics['mean_squared_error'] = mean_squared_error
rmetrics['rmse'] = rmse

In [ ]:
# drop null values

rdata = data[variables]

xtrain, xtest, ytrain, ytest = train_test_split(
    rdata.drop(target, axis=1), rdata[target], random_state=0)


In [ ]:
# apply LinearRegression and predict

# apply
lr = LinearRegression()
lr.fit(xtrain, ytrain)

# predict
ypred = lr.predict(xtest)

In [ ]:
# explicit metric calculation calculation

r2 = 1 - np.sum(np.square(ytest - ypred)) / np.sum(np.square(ytest - np.mean(ytest)))
rmse = np.sqrt(np.mean(np.square(ytest - ypred)))

print(f'R-squared: {r2:.2f}')
print(f'RMSE:      {rmse:.2f}')

In [ ]:
def regression_wrapper(xtrain, xtest, ytrain, ytest, data, show=True):
    '''
    Convenience function wrapping:
    application of linear regression
    presentation of metrics and
    plotting the results
    '''

    # fit the linear regression model
    lr = LinearRegression()
    lr.fit(xtrain, ytrain)

    # apply the model
    ypred = lr.predict(xtest)

    # print and plot results
    if show:
        print('*** model paramters:')
        print('coeff.: ', ', '.join([f'{x:.3f}' for x in lr.coef_]))
        print(f'inter.: {lr.intercept_:.3f}')
        print()
        print('*** scores:')
        for k, v in rmetrics.items():
            score = v(ytest, ypred)
            print(f'{k:22} {score:.3f}')

        plot = sns.jointplot(data=data, x='square_meter', y='price', marker='.', marginal_kws=dict(bins=25));
        plot.ax_joint.plot(xtest, ypred, '-', color='violet' );
    return ypred

In [ ]:
# fit and plot

ypred = regression_wrapper(xtrain, xtest, ytrain, ytest, rdata)

Repeat the above analysis but drop imputed values

REWRITE:\
The integer columns **z_score_price** and **z_score_square_meter**\
indicate outliers with a zscore > 3 with 1\
(otherwise 0).


Tips:\
Sum up the zscores: z_score_price and z_score_square_meter\
Create a boolean mask for zscores that sum up larger than 1\
Apply the mask and drop nans: using where and dropna

In [ ]:
# drop the imputed variables

pattern = '^imp.*({}|{})$'.format(*variables)
mask = data.filter(regex=pattern, axis=1).sum(axis=1) < 1

rdata = rdata.where(mask).dropna()

In [ ]:
# train-test split

xtrain, xtest, ytrain, ytest = train_test_split(
    rdata.drop(target, axis=1), rdata[target], random_state=0)

# fit and plot

ypred = regression_wrapper(xtrain, xtest, ytrain, ytest, rdata)

Log transform the variables using: np.log

In [ ]:
# log transformation

rdata = np.log(rdata+1)

In [ ]:
# train-test split
xtrain, xtest, ytrain, ytest = train_test_split(
    rdata.drop(target, axis=1), rdata[target], random_state=0)

# fit model / plot results again
ypred = regression_wrapper(xtrain, xtest, ytrain, ytest, rdata)

Unlog

In [ ]:
# unlog

ypred_exp = np.exp(ypred) - 1
ytest_exp = np.exp(ytest) - 1

r2 = 1 - np.sum(np.square( ypred_exp - ytest_exp )) / np.sum(np.square( ytest_exp.mean() - ytest_exp ))
rmse = np.sqrt(np.mean(np.square(ypred_exp - ytest_exp)))

print(f'r2   = {r2:.2f}')
print(f'rmse = {rmse:.2f}')

fig, ax = plt.subplots();
sns.scatterplot(data=np.exp(rdata)-1, x='square_meter', y='price', ax=ax);
ax.scatter(np.exp(xtest)-1, np.exp(ypred)-1, color='violet');

Normalize

In [ ]:
# normalize the log data

rdata = (rdata - rdata.mean()) / rdata.std()

In [ ]:
# train-test split
xtrain, xtest, ytrain, ytest = train_test_split(
    rdata.drop(target, axis=1), rdata[target], random_state=0)

# fit model / plot results again
ypred = regression_wrapper(xtrain, xtest, ytrain, ytest, rdata)

In [ ]:
# residuals with:
# LOESS (locally estimated scatterplot smoothing)

tmp = rdata[variables]
tmp['residuals'] = (ytest - ypred)

fig, ax = plt.subplots()
sns.residplot(data=tmp, x='square_meter', y='residuals', lowess=True, line_kws=dict(color='red'), ax=ax);
ax.axis('equal');


# Evaluate the R2

- r2 < 0.3 is considered a None or Very weak effect size,

- 0.3 < r2 < 0.5 is considered a weak or low effect size,

- 0.5 < r2 < 0.7 is considered a Moderate effect size,

- 0.7 < r2 < 1.0 is considered strong effect size,

---

# Linear Regression with Multiple Features

## Feature Selection

In [ ]:
# correlation

cor = np.abs(data.drop(get_dichotomous(data), axis=1).corr())

# absolute correlation
fig, axs = plt.subplots(1, 2, figsize=(16, 6))
sns.heatmap(cor, annot=False, cmap=plt.cm.Blues, vmin=0, vmax=1, ax=axs[0]);

# absolution correlation > 0.7
sns.heatmap(cor.where(cor>0.7, other=0), annot=False, cmap=plt.cm.Blues, vmin=0, vmax=1, ax=axs[1]);

In [ ]:
# select only numberc data

target = 'price'

rdata = data.select_dtypes(include=[np.number])

In [ ]:
# forward feature selection

x = rdata.drop(target, axis=1)
y = rdata[target]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0)

# forward feature selection
print('*** selected features:')
max_features = 16
features = []
for i in range(1, max_features):
    estimator = LinearRegression()
    #estimator = RandomForestRegressor()
    selector = SelectFromModel(estimator, max_features=i).fit(xtrain, ytrain)

    # Only keep the best columns
    mask = selector.get_support()
    cnames = xtrain.columns[mask]
    features.append(cnames)

    print(i, ', '.join(list(cnames)))

In [ ]:
# linear regression for all the feature sets

r2s = []
for feature in features:
    variables = list(feature) + [target]
    tmp = rdata[variables]

    x = tmp.drop(target, axis=1)
    y = tmp[target]

    xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0)

    lr = LinearRegression()
    #lr = Lasso()
    #lr = RandomForestRegressor()
    lr.fit(xtrain, ytrain)

    ypred = lr.predict(xtest)
    r2 = r2_score(ytest, ypred)

    # calculate r2 adjusted
    n = np.shape(ytest)[0]
    k = len(feature) + 1
    r2_adj = 1 - (1 - r2) * (n - 1) / (n - k)
    
    print(', '.join(variables))
    print(f'r2={r2:.3f} f2_adj={r2_adj:.3f}')
    print()
    r2s.append(r2_adj)


x = rdata.drop(target, axis=1)
y = rdata[target]
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0)

lr = LinearRegression()
#lr = Lasso()
#lr = RandomForestRegressor()
lr.fit(xtrain, ytrain)

ypred = lr.predict(xtest)
r2 = r2_score(ytest, ypred)

# calculate r2 adjusted
n = np.shape(ytest)[0]
k = len(rdata.columns)
r2_adj = 1 - (1 - r2) * (n - 1) / (n - k)

print(', '.join(variables))
print(f'r2={r2:.3f} f2_adj={r2_adj:.3f}')
print()
r2s.append(r2_adj)

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(r2s)
ax.set_title('$r^2_{adj}$');

In [ ]:
# log transformation on data skew

# exclude categorical data
tmp = data.select_dtypes(include=[np.number])
tmp = tmp.drop(get_dichotomous(tmp), axis=1)

skew = pd.DataFrame(tmp.skew(), columns=['skew'])
skew['log_skew'] = np.log(tmp + 1).skew()
skew['log_skew/skew'] = np.abs(skew['log_skew'] / skew['skew'] * 100)

display(skew.sort_values('log_skew/skew'))

# columns: 40% less skew from log
mask = skew['log_skew/skew'] < 60
log_columns = skew[mask].index

In [ ]:
# apply log transform

# drop old variables
rdata = rdata.drop(log_columns, axis=1)

# create new log variables
new_log_columns = [f'log_{v}' for v in log_columns]
rdata[new_log_columns] = np.log(data[log_columns] + 1)

target = 'log_price'


In [ ]:
# forward feature selection

x = rdata.drop(target, axis=1)
y = rdata[target]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0)

# forward feature selection
print('*** selected features:')
max_features = 16
features = []
for i in range(1, max_features):
    estimator = LinearRegression()
    #estimator = RandomForestRegressor()
    selector = SelectFromModel(estimator, max_features=i).fit(xtrain, ytrain)

    # Only keep the best columns
    mask = selector.get_support()
    cnames = xtrain.columns[mask]
    features.append(cnames)

    print(i, ', '.join(list(cnames)))

In [ ]:
# linear regression for all the feature sets

r2s_log = []
for feature in features:
    variables = list(feature) + [target]
    tmp = rdata[variables]

    x = tmp.drop(target, axis=1)
    y = tmp[target]

    xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0)

    lr = LinearRegression()
    #lr = Lasso()
    #lr = RandomForestRegressor()
    lr.fit(xtrain, ytrain)

    ypred = lr.predict(xtest)
    r2 = r2_score(np.exp(ytest)-1, np.exp(ypred)-1)

    # calculate r2 adjusted
    n = np.shape(ytest)[0]
    k = len(feature) + 1
    r2_adj = 1 - (1 - r2) * (n - 1) / (n - k)
    
    print(', '.join(variables))
    print(f'r2={r2:.3f} f2_adj={r2_adj:.3f}')
    print()
    r2s_log.append(r2_adj)


x = rdata.drop(target, axis=1)
y = rdata[target]
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=0)

lr = LinearRegression()
#lr = Lasso()
#lr = RandomForestRegressor()
lr.fit(xtrain, ytrain)

ypred = lr.predict(xtest)
r2 = r2_score(np.exp(ytest)-1, np.exp(ypred)-1)

# calculate r2 adjusted
n = np.shape(ytest)[0]
k = len(rdata.columns)
r2_adj = 1 - (1 - r2) * (n - 1) / (n - k)

print(', '.join(variables))
print(f'r2={r2:.3f} f2_adj={r2_adj:.3f}')
print()
r2s_log.append(r2_adj)



In [ ]:
# the last data point jumps and contains all features

fig, axs = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

axs[0].plot(r2s)
axs[0].set_title('$r^2_{adj}$');

axs[1].plot(r2s_log)
axs[1].set_title('$r^2_{adj}$ with log');

In [ ]:
# standardize (makes results worse)

#norm_columns = rdata.drop(get_dichotomous(tmp), axis=1).columns
#rdata[norm_columns] = (rdata[norm_columns] - rdata[norm_columns].mean()) / rdata[norm_columns].std()

---
---
---